In [4]:
import requests
from bs4 import BeautifulSoup

import requests
from urllib.parse import urlparse, urljoin
from bs4 import BeautifulSoup
import requests
from urllib.parse import urlparse, urljoin
from bs4 import BeautifulSoup
import logging

# Set up basic logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

class WebCrawler:
    def __init__(self, base_url):
        self.base_url = base_url
        self.base_domain = urlparse(base_url).netloc
        self.visited_urls = set()
        self.texts = []

    def crawl(self, url):
        print(url)
        # Normalize the URL by removing the fragment
        url = urlparse(url)._replace(fragment="").geturl()

        # Ensure the URL has not been visited and is within the same domain
        if url in self.visited_urls or urlparse(url).netloc != self.base_domain:
            return
        logging.info(f"Crawling: {url}")
        self.visited_urls.add(url)

        # Send a GET request to the URL
        try:
            response = requests.get(url, timeout=5)
            response.raise_for_status()
        except requests.RequestException as e:
            logging.error(f"Request failed: {e}")
            return

        # Parse the HTML content of the page
        soup = BeautifulSoup(response.text, 'html.parser')

        # Extract and store the text content of the page
        text = soup.get_text(separator=' ', strip=True)
        self.texts.append({'url': url, 'text': text})

        # Find all <a> tags, then recursively crawl each href found
        for link in soup.find_all('a', href=True):
            href = link['href']
            if not href.startswith('http'):
                href = urljoin(url, href)
            self.crawl(href)

    def get_site_text(self):
        # Begin crawling from the base URL
        self.crawl(self.base_url)
        return self.texts

# Example usage
crawler = WebCrawler("https://vik.wiki/")
site_texts = crawler.get_site_text()
# For demonstration purposes, let's just print the URLs collected
for page in site_texts:
    logging.info(f"URL: {page['url']}, Text Length: {len(page['text'])}")


2024-03-19 17:53:52,908 - INFO - Crawling: https://vik.wiki/


https://vik.wiki/


2024-03-19 17:53:58,171 - ERROR - Request failed: HTTPSConnectionPool(host='vik.wiki', port=443): Read timed out. (read timeout=5)
